In [43]:
import heapq
from collections import Counter
from itertools import product
from math import prod
import numpy as np
from dataclasses import dataclass
import cvxpy as cp

In [44]:
#FILE = "example.txt"
FILE = "input.txt"

In [45]:
inputs = [
    line.rstrip('\n').split(' ') for line in open(FILE)
]

In [46]:
class Machine:
    lights: str
    lights_bitmask: np.ndarray
    wiring: list[list[int]]
    wiring_bitmasks: np.ndarray
    joltages: list[int]
    joltages_np: np.ndarray
    required_permutations: int

    def __init__(self, lights: str, wiring: list[list[int]], joltages: list[int]):
        self.lights = lights
        self.lights_bitmask = np.array(
            [1 if i == '#' else 0 for i in lights]
        )
        self.wiring = wiring
        self.joltages = joltages
        self.joltages_np = np.array(joltages)

        self.wiring_bitmasks = np.array([
            [1 if i in j else 0 for i in range(len(lights))] for j in wiring
        ])

        self.required_permutations = self.get_required_permutations()
    
    def get_required_permutations(self):

            n_switches = len(self.wiring)
            
            x = cp.Variable(n_switches, integer=True)
            obj = cp.Minimize(cp.sum(x))
            constraints = [
                self.wiring_bitmasks.T @ x == self.joltages_np,
                x >= 0,
            ]
            prob = cp.Problem(obj, constraints)
            prob.solve()

            return int(prob.value)

In [47]:
parsed_inputs = [
    Machine(
        lights=line[0].lstrip('[').rstrip(']'),
        wiring=[
            list(
                map(
                    int, i.lstrip('(').rstrip(')').strip().split(',')
                )
            ) 
             for i in line[1:-1]
        ],
        joltages=tuple(map(int, line[-1].strip('{').strip('}').split(',')))
    ) for line in inputs
    ]

In [48]:
parsed_inputs[0].wiring

[[2, 3, 4, 5], [0, 2, 3, 5, 6], [1, 4, 6], [2, 3, 4], [0, 5]]

In [49]:
parsed_inputs[0].wiring_bitmasks

array([[0, 0, 1, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 1, 1],
       [0, 1, 0, 0, 1, 0, 1],
       [0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 0]])

In [50]:
parsed_inputs[0].lights_bitmask

array([0, 0, 1, 1, 1, 1, 0])

In [51]:
parsed_inputs[0].required_permutations

47

In [52]:
required_permutations = sum(i.required_permutations for i in parsed_inputs)

In [53]:
required_permutations

15631